[Kaggle NLP Getting Started Tutorial](https://www.kaggle.com/code/philculliton/nlp-getting-started-tutorial/notebook)

## NLP Tutorial
NLP - or Natural Language Processing - is shorthand for a wide array of techniques designed to help machines learn from text. Natural Language Processing powers everything from chatbots to search engines, and is used in diverse tasks like sentiment analysis and machine translation.

In this tutorial we'll look at this competition's dataset, use a simple technique to process it, build a machine learning model, and submit predictions for a score!

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [4]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

## A quick look at our data
Let's look at our data... first, an example of what is NOT a disaster tweet.

In [5]:
train_df[train_df["target"] == 0]["text"].values[1]

'I love fruits'

And one that is:

In [6]:
train_df[train_df["target"] == 1]["text"].values[1]

'Forest fire near La Ronge Sask. Canada'

## Building vectors
The theory behind the model we'll build in this notebook is pretty simple: the words contained in each tweet are a good indicator of whether they're about a real disaster or not (this is not entirely correct, but it's a great place to start).

We'll use scikit-learn's CountVectorizer to count the words in each tweet and turn them into data our machine learning model can process.

Note: a vector is, in this context, a set of numbers that a machine learning model can work with. We'll look at one in just a second.

In [7]:
count_vectorizer = feature_extraction.text.CountVectorizer()

## let's get counts for the first 5 tweets in the data
example_train_vectors = count_vectorizer.fit_transform(train_df["text"][0:5])

In [8]:
## we use .todense() here because these vectors are "sparse" (only non-zero elements are kept to save space)
print(example_train_vectors[0].todense().shape)
print(example_train_vectors[0].todense())

(1, 54)
[[0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0]]


The above tells us that:

1. There are 54 unique words (or "tokens") in the first five tweets.
2. The first tweet contains only some of those unique tokens - all of the non-zero counts above are the tokens that DO exist in the first tweet.

Now let's create vectors for all of our tweets.

In [17]:
train_vectors = count_vectorizer.fit_transform(train_df["text"])

## note that we're NOT using .fit_transform() here. Using just .transform() makes sure
# that the tokens in the train vectors are the only ones mapped to the test vectors - 
# i.e. that the train and test vectors use the same set of tokens.
test_vectors = count_vectorizer.transform(test_df["text"])
print(f"test_vectors: {test_vectors}")

test_vectors: <Compressed Sparse Row sparse matrix of dtype 'int64'
	with 41511 stored elements and shape (3263, 21637)>
  Coords	Values
  (0, 3904)	1
  (0, 4977)	1
  (0, 8753)	1
  (0, 10550)	1
  (0, 18591)	1
  (1, 1464)	1
  (1, 4357)	1
  (1, 5783)	1
  (1, 6379)	1
  (1, 6934)	1
  (1, 8893)	1
  (1, 10043)	1
  (1, 16503)	1
  (1, 17843)	1
  (2, 1539)	1
  (2, 1851)	1
  (2, 2192)	1
  (2, 2358)	1
  (2, 3864)	1
  (2, 7439)	1
  (2, 7524)	1
  (2, 7652)	1
  (2, 10043)	1
  (2, 14751)	1
  (2, 16637)	1
  :	:
  (3259, 21164)	1
  (3260, 4232)	1
  (3260, 4517)	1
  (3260, 5624)	1
  (3260, 8448)	1
  (3260, 9304)	1
  (3260, 9718)	1
  (3260, 11479)	1
  (3261, 658)	1
  (3261, 4517)	1
  (3261, 8840)	1
  (3261, 9304)	1
  (3261, 9396)	1
  (3261, 10070)	1
  (3261, 12239)	1
  (3261, 14021)	1
  (3261, 20439)	1
  (3262, 1550)	1
  (3262, 4364)	1
  (3262, 6609)	1
  (3262, 8801)	1
  (3262, 10084)	1
  (3262, 12866)	1
  (3262, 14619)	1
  (3262, 21363)	1


## Our model
As we mentioned above, we think the words contained in each tweet are a good indicator of whether they're about a real disaster or not. The presence of particular word (or set of words) in a tweet might link directly to whether or not that tweet is real.

What we're assuming here is a linear connection. So let's build a linear model and see!

In [10]:
## Our vectors are really big, so we want to push our model's weights
## toward 0 without completely discounting different words - ridge regression 
## is a good way to do this.
clf = linear_model.RidgeClassifier()

Let's test our model and see how well it does on the training data. For this we'll use `cross-validation` - where we train on a portion of the known data, then validate it with the rest. If we do this several times (with different portions) we can get a good idea for how a particular model or method performs.

The metric for this competition is F1, so let's use that here.

In [19]:
scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=3, scoring="f1")
scores

array([0.59453669, 0.5642787 , 0.64082434])

The above scores aren't terrible! It looks like our assumption will score roughly 0.65 on the leaderboard. There are lots of ways to potentially improve on this (TFIDF, LSA, LSTM / RNNs, the list is long!) - give any of them a shot!

In the meantime, let's do predictions on our training set and build a submission for the competition.

In [20]:
clf.fit(train_vectors, train_df["target"])

,alpha,1.0
,fit_intercept,True
,copy_X,True
,max_iter,None
,tol,0.0001
,class_weight,None
,solver,'auto'
,positive,False
,random_state,None


In [21]:
sample_submission = pd.read_csv("data/sample_submission.csv")